In [2]:
import numpy as np
import matplotlib.pyplot as plt
import xarray as xr

from xmitgcm import open_mdsdataset

import pych
import pych.ecco as pe

from smooth_store import open_smoothdataset

In [3]:
from new_smooth_store import open_smoothdataset
#import os
#from smooth_store import get_smooth_vars
#import xmitgcm
#def open_smoothdataset(run_dir,
#                       read_filternorm=False,
#                       **kwargs):
#    """
#    TODO:
#        - read_filternorm necessary for OIDriver still?
#        - split up different operators, remove number from variable name (add as dimension)
#        - add 2D variables... etc
#    """
#
#    # get all smooth*.data prefixes, then their operator numbers
#    prefixes = [x for x in os.listdir(run_dir) if 'smooth' in x and '.data' in x]
#    numbers = [x.split('.')[0][-3:] for x in prefixes]
#
#    # Right now operators will be a list of strings, since going to want string not int anyway...
#    operators = []
#    for x in numbers:
#        if x not in operators:
#            operators.append(x)
#
#    extras = [get_smooth_vars(op) for op in operators]
#    custom_grid_variables = {}
#    extra_variables = {}
#
#    # Have to loop to get fields for each operator
#    for grid_extras, var_extras in extras:
#        custom_grid_variables.update(grid_extras)
#        extra_variables.update(var_extras)
#        
#    xds = xmitgcm.open_mdsdataset(run_dir, custom_grid_variables=custom_grid_variables, extra_variables=extra_variables, **kwargs)
#
#    # Move all smooth variables to data_vars status
#    smoothvars = [vname for vname in xds.coords if 'smooth' in vname]
#    xds = xds.reset_coords(smoothvars)
#    
#    # These get read in with a "time" dimension, make this "sample" 
#    xds['sample'] = xr.DataArray(np.arange(len(xds['time'])), coords=xds['time'].coords, dims=xds['time'].dims)
#    
#    xds = xds.set_coords('sample').swap_dims({'time':'sample'})
#    xds = xds.drop(['time','iter'])
#    return xds

In [5]:
run_dir='/scratch2/tsmith/generic-matern-covariance/sampling/llc90/testwrite-3D-C/run.10dx.01xi/'
#grid_dir='/scratch2/tsmith/generic-matern-covariance/sampling/llc90/grid'
ds = open_mdsdataset(run_dir,
                     iters=None,
                     geometry='llc')

/home/tsmith/xmitgcm/xmitgcm/mds_store.py:922: UserWarning: Couldn't find available_diagnostics.log in /scratch2/tsmith/generic-matern-covariance/sampling/llc90/testwrite-3D-C/run.10dx.01xi/ or /scratch2/tsmith/generic-matern-covariance/sampling/llc90/testwrite-3D-C/run.10dx.01xi/. Using default version.
  warnings.warn("Couldn't find available_diagnostics.log "


In [10]:
ds =open_smoothdataset(run_dir, prefix='smooth3Dfld001', geometry='llc', llc_method='bigchunks')

/home/tsmith/xmitgcm/xmitgcm/mds_store.py:922: UserWarning: Couldn't find available_diagnostics.log in /scratch2/tsmith/generic-matern-covariance/sampling/llc90/testwrite-3D-C/run.10dx.01xi/ or /scratch2/tsmith/generic-matern-covariance/sampling/llc90/testwrite-3D-C/run.10dx.01xi/. Using default version.
  warnings.warn("Couldn't find available_diagnostics.log "


In [ ]:
ds.ginv_norm.load()

In [ ]:
ds.ginv_norm